# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [2]:
# load in the data
azdias = pd.read_csv('Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('Udacity_CUSTOMERS_052018.csv', sep=';')

# delete empty columns
del azdias['Unnamed: 0']
del customers['Unnamed: 0']

C:\Users\Tomas\PycharmProjects\nano_degree_DS\venv\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
azdias.head()

LNR  AGER_TYP  AKT_DAT_KL  ALTER_HH  ALTER_KIND1  ALTER_KIND2  \
0  910215        -1         NaN       NaN          NaN          NaN   
1  910220        -1         9.0       0.0          NaN          NaN   
2  910225        -1         9.0      17.0          NaN          NaN   
3  910226         2         1.0      13.0          NaN          NaN   
4  910241        -1         1.0      20.0          NaN          NaN   

   ALTER_KIND3  ALTER_KIND4  ALTERSKATEGORIE_FEIN  ANZ_HAUSHALTE_AKTIV  \
0          NaN          NaN                   NaN                  NaN   
1          NaN          NaN                  21.0                 11.0   
2          NaN          NaN                  17.0                 10.0   
3          NaN          NaN                  13.0                  1.0   
4          NaN          NaN                  14.0                  3.0   

          ...           VHN  VK_DHT4A  VK_DISTANZ  VK_ZG11  W_KEIT_KIND_HH  \
0         ...           NaN       NaN         NaN      NaN             NaN   
1         ...           4.0       8.0        11.0     10.0             3.0   
2         ...           2.0       9.0         9.0      6.0             3.0   
3         ...           0.0       7.0        10.0     11.0             NaN   
4         ...           2.0       3.0         5.0      4.0             2.0   

   WOHNDAUER_2008  WOHNLAGE ZABEOTYP ANREDE_KZ ALTERSKATEGORIE_GROB  
0             NaN       NaN        3         1                    2  
1             9.0       4.0        5         2                    1  
2             9.0       2.0        5         2                    3  
3             9.0       7.0        3         2                    4  
4             9.0       3.0        4         1                    3  

[5 rows x 366 columns]

In [4]:
customers.head()

LNR  AGER_TYP  AKT_DAT_KL  ALTER_HH  ALTER_KIND1  ALTER_KIND2  \
0    9626         2         1.0      10.0          NaN          NaN   
1    9628        -1         9.0      11.0          NaN          NaN   
2  143872        -1         1.0       6.0          NaN          NaN   
3  143873         1         1.0       8.0          NaN          NaN   
4  143874        -1         1.0      20.0          NaN          NaN   

   ALTER_KIND3  ALTER_KIND4  ALTERSKATEGORIE_FEIN  ANZ_HAUSHALTE_AKTIV  \
0          NaN          NaN                  10.0                  1.0   
1          NaN          NaN                   NaN                  NaN   
2          NaN          NaN                   0.0                  1.0   
3          NaN          NaN                   8.0                  0.0   
4          NaN          NaN                  14.0                  7.0   

          ...           VK_ZG11  W_KEIT_KIND_HH  WOHNDAUER_2008  WOHNLAGE  \
0         ...               2.0             6.0             9.0       7.0   
1         ...               3.0             0.0             9.0       NaN   
2         ...              11.0             6.0             9.0       2.0   
3         ...               2.0             NaN             9.0       7.0   
4         ...               4.0             2.0             9.0       3.0   

   ZABEOTYP      PRODUCT_GROUP  CUSTOMER_GROUP ONLINE_PURCHASE ANREDE_KZ  \
0         3  COSMETIC_AND_FOOD     MULTI_BUYER               0         1   
1         3               FOOD    SINGLE_BUYER               0         1   
2         3  COSMETIC_AND_FOOD     MULTI_BUYER               0         2   
3         1           COSMETIC     MULTI_BUYER               0         1   
4         1               FOOD     MULTI_BUYER               0         1   

  ALTERSKATEGORIE_GROB  
0                    4  
1                    4  
2                    4  
3                    4  
4                    3  

[5 rows x 369 columns]

**DataFrame sizes**

In [5]:
print("AZDIAS dataframe size\nrows: {}\ncols: {}".format(*azdias.shape))

AZDIAS dataframe size
rows: 891221
cols: 366


In [6]:
print("CUSTOMERS dataframe size\nrows: {}\ncols: {}".format(*customers.shape))

CUSTOMERS dataframe size
rows: 191652
cols: 369


### 0.1 Load DataFrame with information about all attributes

1. read `DIAS Attributes - Values 2017.xlsx` file into pandas DataFrame,
2. iterate over rows, read each attributes name, description and value meanings,
3. read `DIAS Information Levels - Attributes 2017.xlsx` file into pandas DataFrame,
4. join both DataFrames on Attribute and Description columns into one DataFrame.

In [7]:
# 1. read `DIAS Attributes - Values 2017.xlsx` file into pandas DataFrame
df_attr_1 = pd.read_excel('DIAS Attributes - Values 2017.xlsx')
del df_attr_1['Unnamed: 0']
df_attr_1.head()

Attribute         Description Value                     Meaning
0  AGER_TYP  best-ager typology    -1                     unknown
1       NaN                 NaN     0  no classification possible
2       NaN                 NaN     1             passive elderly
3       NaN                 NaN     2            cultural elderly
4       NaN                 NaN     3   experience-driven elderly

In [8]:
# 2. iterate over rows, read each attributes name, description and value meanings,
rows = list()
start = False
for idx in df_attr_1.index:
    if df_attr_1.iloc[idx].Attribute is not np.nan:
        if start:
            rows.append([attribute, description, values, meanings])
        # create lists to store Values and Meanings
        values = list()
        meanings = list()
        # read data
        attribute = df_attr_1.iloc[idx].Attribute
        description = df_attr_1.iloc[idx].Description
        # just quick and ungly way to skip saving first row
        start = True
        
    # add values and meanings to the lists
    values.append(df_attr_1.iloc[idx].Value)
    meanings.append(df_attr_1.iloc[idx].Meaning)
    
rows.append([attribute, description, values, meanings])

df_attr_1 = pd.DataFrame(rows, columns=["Attribute", "Description", "Values", "Meanings"])
df_attr_1.head()

Attribute                                   Description  \
0              AGER_TYP                            best-ager typology   
1  ALTERSKATEGORIE_GROB  age classification through prename analysis    
2              ALTER_HH                 main age within the household   
3             ANREDE_KZ                                        gender   
4   ANZ_HAUSHALTE_AKTIV          number of households in the building   

                                              Values  \
0                                   [-1, 0, 1, 2, 3]   
1                             [-1, 0, 1, 2, 3, 4, 9]   
2  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
3                                      [-1, 0, 1, 2]   
4                                                […]   

                                            Meanings  
0  [unknown, no classification possible, passive ...  
1  [unknown, < 30 years, 30 - 45 years, 46 - 60 y...  
2  [unknown / no main age detectable, 01.01.1895 ...  
3                            [unknown, male, female]  
4        [numeric value (typically coded from 1-10)]

In [9]:
# 3. read DIAS Information Levels - Attributes 2017.xlsx file into pandas DataFrame
df_attr_2 = pd.read_excel('DIAS Information Levels - Attributes 2017.xlsx')
del df_attr_2['Unnamed: 0']
df_attr_2.head()

Information level             Attribute  \
0               NaN              AGER_TYP   
1            Person  ALTERSKATEGORIE_GROB   
2               NaN             ANREDE_KZ   
3               NaN         CJT_GESAMTTYP   
4               NaN     FINANZ_MINIMALIST   

                                         Description  \
0                                 best-ager typology   
1                      age through prename analysis    
2                                             gender   
3  Customer-Journey-Typology relating to the pref...   
4         financial typology: low financial interest   

                                    Additional notes  
0  in cooperation with Kantar TNS; the informatio...  
1  modelled on millions of first name-age-referen...  
2                                                NaN  
3  relating to the preferred information, marketi...  
4  Gfk-Typology based on a representative househo...

There are 5 attribute names which were written in inconvienient manner:

* `D19_GESAMT_ANZ_12                                    D19_GESAMT_ANZ_24`,
* `D19_BANKEN_ ANZ_12             D19_BANKEN_ ANZ_24`,
* `D19_TELKO_ ANZ_12                  D19_TELKO_ ANZ_24`,
* `D19_VERSI_ ANZ_12                                       D19_VERSI_ ANZ_24`,
* `D19_VERSAND_ ANZ_12          D19_VERSAND_ ANZ_24`.

These attributes will be manually fixed in cells bellow.

In [10]:
attr_list = ["D19_GESAMT_ANZ_12                                    D19_GESAMT_ANZ_24",
            "D19_BANKEN_ ANZ_12             D19_BANKEN_ ANZ_24",
            "D19_TELKO_ ANZ_12                  D19_TELKO_ ANZ_24",
            "D19_VERSI_ ANZ_12                                       D19_VERSI_ ANZ_24",
            "D19_VERSAND_ ANZ_12          D19_VERSAND_ ANZ_24"]

# get rows of attributes of interest into new DataFrame
df_attr_3 = df_attr_2[df_attr_2.Attribute.isin(attr_list)]

# remove those attribrutes from DataFrame 
df_attr_2 = df_attr_2[~df_attr_2.Attribute.isin(attr_list)]

# check new DataFrame
df_attr_3

Information level                                          Attribute  \
50               NaN  D19_GESAMT_ANZ_12                             ...   
51               NaN  D19_BANKEN_ ANZ_12             D19_BANKEN_ ANZ_24   
52               NaN  D19_TELKO_ ANZ_12                  D19_TELKO_ ...   
53               NaN  D19_VERSI_ ANZ_12                             ...   
54               NaN   D19_VERSAND_ ANZ_12          D19_VERSAND_ ANZ_24   

                                          Description Additional notes  
50  transaction activity TOTAL POOL in the last 12...              NaN  
51  transaction activity BANKS in the last 12 and ...              NaN  
52  transaction activity TELCO in the last 12 and ...              NaN  
53  transaction activity INSURANCE in the last 12 ...              NaN  
54  transaction activity MAIL-ORDER in the last 12...              NaN

Creating a function which splits string into new strings was not an option because `D19_GESAMT_ANZ_12                                    D19_GESAMT_ANZ_24` would have two new strings `D19_GESAMT_ANZ_12` and `D19_GESAMT_ANZ_24`, but other attributes like `D19_BANKEN_ ANZ_12             D19_BANKEN_ ANZ_24` would result in 4 new strings `D19_BANKEN_`, `ANZ_12`, `D19_BANKEN_` and `ANZ_24`. Therefore I chose to just manualy iterate over rows and resave them into list.

In [11]:
# temprarry list to store data
_data = list()

for idx in df_attr_3.index:
    # split decription value with large space
    attributes = df_attr_3.loc[idx].Attribute.split("          ")
    # remove unneccesary spaces
    attribute_12 = attributes[0].replace(" ", "")
    attribute_24 = attributes[-1].replace(" ", "")
    
    # split decription value with " and "
    description = df_attr_3.loc[idx].Description.split(" and ")
    # create new descriptions for 12 and 24 months
    description_12 = description[0] + " months"
    description_24 = description[0][:-2] + description[1]
    
    # add information about 12 month attribute
    _data.append([np.nan, attribute_12, description_12, np.nan])
    # add information about 24 month attribute
    _data.append([np.nan, attribute_24, description_24, np.nan])
    
    
# resave data into DataFrame
df_attr_3 = pd.DataFrame(_data, columns=df_attr_3.columns)
df_attr_3

Information level           Attribute  \
0                NaN   D19_GESAMT_ANZ_12   
1                NaN   D19_GESAMT_ANZ_24   
2                NaN   D19_BANKEN_ANZ_12   
3                NaN   D19_BANKEN_ANZ_24   
4                NaN    D19_TELKO_ANZ_12   
5                NaN    D19_TELKO_ANZ_24   
6                NaN    D19_VERSI_ANZ_12   
7                NaN    D19_VERSI_ANZ_24   
8                NaN  D19_VERSAND_ANZ_12   
9                NaN  D19_VERSAND_ANZ_24   

                                         Description  Additional notes  
0  transaction activity TOTAL POOL in the last 12...               NaN  
1  transaction activity TOTAL POOL in the last 24...               NaN  
2   transaction activity BANKS in the last 12 months               NaN  
3   transaction activity BANKS in the last 24 months               NaN  
4   transaction activity TELCO in the last 12 months               NaN  
5   transaction activity TELCO in the last 24 months               NaN  
6  transaction activity INSURANCE in the last 12 ...               NaN  
7  transaction activity INSURANCE in the last 24 ...               NaN  
8  transaction activity MAIL-ORDER in the last 12...               NaN  
9  transaction activity MAIL-ORDER in the last 24...               NaN

In [12]:
# 4. join all DataFrames on Attribute and Description columns into one DataFrame.
df_attr = pd.merge(df_attr_2, df_attr_3, on=['Attribute'], how='outer')
df_attr = pd.merge(df_attr_1, df_attr, on=['Attribute'], how='outer')
# rename Description_x columns
# some descriptions for the same attribute differs in DataFrames, hence both descriptions are saved as separate columns
df_attr = df_attr.rename(columns={'Description_x': 'Description_a', 'Description_y': 'Description_i'})
df_attr = df_attr.reset_index(drop=True)
df_attr.head()

Attribute                                   Description  \
0              AGER_TYP                            best-ager typology   
1  ALTERSKATEGORIE_GROB  age classification through prename analysis    
2              ALTER_HH                 main age within the household   
3             ANREDE_KZ                                        gender   
4   ANZ_HAUSHALTE_AKTIV          number of households in the building   

                                              Values  \
0                                   [-1, 0, 1, 2, 3]   
1                             [-1, 0, 1, 2, 3, 4, 9]   
2  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
3                                      [-1, 0, 1, 2]   
4                                                […]   

                                            Meanings Information level_x  \
0  [unknown, no classification possible, passive ...                 NaN   
1  [unknown, < 30 years, 30 - 45 years, 46 - 60 y...              Person   
2  [unknown / no main age detectable, 01.01.1895 ...           Household   
3                            [unknown, male, female]                 NaN   
4        [numeric value (typically coded from 1-10)]            Building   

                                 Description_a  \
0                           best-ager typology   
1                age through prename analysis    
2                main age within the household   
3                                       gender   
4  number of households known in this building   

                                  Additional notes_x  Information level_y  \
0  in cooperation with Kantar TNS; the informatio...                  NaN   
1  modelled on millions of first name-age-referen...                  NaN   
2                                                NaN                  NaN   
3                                                NaN                  NaN   
4                                                NaN                  NaN   

  Description_i  Additional notes_y  
0           NaN                 NaN  
1           NaN                 NaN  
2           NaN                 NaN  
3           NaN                 NaN  
4           NaN                 NaN

### 0.2 Check for missing values

First identify attributes, which have NAN values. Iterate over all columns and count procentage of missing values in each collumn.

In [ ]:
def convert_dtype(notation):
    """
    Converts 
    """

In [ ]:
data_list = list()

# for col in azdias.columns:
for col in df_attr.Attribute:
    # get information about particular attribute
    _df = df_attr[df_attr.Attribute == col]
    # try iterating over Meanings values
    try:
        len(_df.Meanings.values[0])
        ok = True
    except:
        ok = False
    
    notations = list()
    notations_2 = list()
    if ok:
        for idx in range(_df.Meanings.values.shape[0]):
            meaning = _df.Meanings.values[0][idx]
            notation = _df.Values.values[0][idx]
            if ("unknown" in meaning) or ('none' in meaning):             
                if isinstance(notation, str):
                    notation = [int(_) for _ in notation.split(',')]
                elif isinstance(notation, int):
                    notation = [notation]      

                notations += notation
                notations_2.append(_df.Meanings.values[0][idx])

    data_list.append([col, notations, notations_2])

In [ ]:
data_list

In [ ]:
df_attr[df_attr.Attribute == "D19_VERSI_ ANZ_12                                       D19_VERSI_ ANZ_24"]

In [ ]:
set(data_types)

In [ ]:
df_attr[df_attr.Attribute == "SEMIO_KULT"].Values.values

In [ ]:
_df.Meanings.values[0]

In [ ]:
_df = df_attr[df_attr.Attribute == "ALTER_HH"]

In [ ]:
for _ in _df.Meanings.values[0]:
    if "unknown" in _:
        print(_)

In [ ]:
for idx in df_attr.index:
    

In [ ]:
df_attr.head()

In [ ]:
for a in df_attr.Attribute.values:
    _df = df_attr[df_attr.Attribute==a]
    if "unknown" in _df.Meanings.values[0]:
        print(_df.Meanings.values)
        


# for _ in df_attr.Meanings:
#     if "unknown" in _:
#         print(_)

In [ ]:
print("Attribute                                | No. of nan")
print("_"*55)
for col in azdias.columns:
    # count missing values
    no_nans = azdias.loc[:, col].isnull().sum()
    if no_nans>0:
        print("{:40s} | {}".format(col, no_nans))
    else:
        print()

In [ ]:
for a in feat_info[feat_info.missing_or_unknown == '[]'].attribute:
    print("{:25s} | {}".format(a, azdias[a].isnull().sum()))

In [ ]:
# Be sure to add in a lot more cells (both markdown and code) to document your
# approach and findings!

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

In [ ]:
mailout_train.to_csv('Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')

In [ ]:
mailout_test.to_csv('Udacity_MAILOUT_052018_TEST.csv', sep=';')